# Exercise 5: The Diffusion Equation — Implicit and Semi-Explicit Methods

## Learning Objectives

By completing this exercise, you will:
- Understand the CFL condition for parabolic (diffusive) PDEs
- Implement explicit, implicit, and semi-explicit time integration schemes
- Apply Newton-Raphson iteration for nonlinear implicit systems
- Compare computational efficiency of different methods
- (Optional) Implement Super-Time-Stepping (STS) acceleration

---

## Introduction

We now consider the viscous (diffusive) term from the full Burgers' equation:

$$\frac{\partial u}{\partial t} = \nu \frac{\partial^2 u}{\partial x^2} \tag{1}$$

This is a **parabolic PDE**, which has very different stability properties compared to the hyperbolic advection equation.

---

## Part 1: Explicit Method

### Task 1.1: Derive the CFL Condition

**Question:** What is the CFL condition for eq. (1) when $\nu$ is:
1. A constant?
2. An array that depends on $x$?

### Task 1.2: Implement Explicit Diffusion

Solve equation (1) numerically with:

**Setup:**
- Domain: $x \in [-2.6, 2.6]$
- Periodic boundary conditions
- Initial condition:
$$u(x,t=t_0) = A\exp\left(-\frac{(x-x_c)^2}{W^2}\right) \tag{2}$$
- Parameters: $A = 0.3$, $W = 0.1$, $x_c = 0$, $\nu = 0.1$

**Implementation:** Fill in `step_diff_burgers` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_5.py).

> **Suggestion:** Apply forward derivative first, then backward derivative. Apply von Neumann analysis.

**Questions:**
1. Is it stable?
2. What is the time-step dependence on $\Delta x$?
3. How many steps are needed to reach $t = 1.8$ for `nump = 128`? And for `nump = 256`?

---

## Choose One of the Following Options:

---

## Option A: Implicit Methods (Newton-Raphson)

Implicit methods can relax the CFL constraint. See the [wiki: Implicit Methods](https://github.com/AST-Course/AST5110/wiki/Implicit-methods) for background.

### Task A.1: Implement the Newton-Rapson Method

The implicit scheme gives a nonlinear system $F(u^{n+1}) = 0$ where:

$$F_j = u^{n+1}_j - u^n_j - \nu \frac{\Delta t}{\Delta x^2}\left(u^{n+1}_{j+1} - 2u^{n+1}_{j} + u^{n+1}_{j-1}\right)$$

**Implementation:** Fill in `NR_f` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_5.py).

### Task A.2: Implement the Jacobian

The Jacobian matrix elements are:

$$J_{jk} = \frac{\partial F_j}{\partial u^{n+1}_k}$$

**Implementation:** Fill in `jacobian` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_5.py).

> **Note:** This matrix is linear in $u$ (tridiagonal structure).

### Task A.3: Validate and Benchmark

1. Test the model against [self-similar solutions](https://github.com/AST-Course/AST5110/wiki/Self-similar-solution-for-parabolic-eq)
2. Compare timing with the Lax method using `time.time`
3. Use: `nump = 256`, `nt = 30`, `dt = 0.1`

### Task A.4: Nonlinear Diffusion (Advanced)

Consider a nonlinear diffusion where $\nu$ depends on $u$:

$$\frac{\partial u}{\partial t} = u \frac{\partial^2 u}{\partial x^2}$$

**Implementation:** Fill in `Newton_Raphson_u`, `jacobian_u`, and `NR_f_u`.

**Parameters:** Error tolerance = $10^{-14}$, increase `ncount` to 1000 if needed.

**Questions:**
1. How many iterations does the method need to converge?
2. Why is this different from the linear case?

---

## Option B: Super-Time-Stepping (STS)

Super-Time-Stepping (STS) is an acceleration technique for parabolic terms. It performs a sequence of "unstable" intermediate steps, but the overall cycle is stable. See the [wiki: Super-Time-Stepping](https://github.com/AST-Course/AST5110/wiki/Super-time-stepping) for details.

### Task B.1: Implement STS

**Implementation:** Fill in `evol_sts` and `taui_sts` in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_5.py).

### Task B.2: Analyze STS Behavior

1. Visualize how `taui_sts` varies with $\nu$ and `niter`
2. Compare the solution with the analytical one for:
   - Final STS step
   - Intermediate STS steps
3. For full STS cycles, how does the solution improve with $\nu$? With `niter`?


### Task B.3: Error Analysis

**Questions:**
1. Is there a relationship between the error and the parameters $\nu$ and `niter`?
2. For which values of `niter` and $\nu$ does STS provide larger effective time steps than ordinary explicit methods?

---

## Hints

<details>
<summary><b>Hint 1: CFL condition for diffusion</b></summary>

For explicit diffusion with centered second derivative:
$$\Delta t \leq \frac{(\Delta x)^2}{2\nu}$$

Note the **quadratic** dependence on $\Delta x$ — this is much more restrictive than hyperbolic equations!
</details>

<details>
<summary><b>Hint 2: Jacobian structure</b></summary>

For the linear diffusion equation, the Jacobian is tridiagonal:
- Diagonal: $1 + 2\nu \Delta t / \Delta x^2$
- Off-diagonal: $-\nu \Delta t / \Delta x^2$

Use sparse matrix operations for efficiency.
</details>

<details>
<summary><b>Hint 3: Newton-Raphson convergence</b></summary>

For linear problems, Newton-Raphson converges in one iteration. For nonlinear problems, expect multiple iterations. Monitor the residual norm to track convergence.
</details>